In [4]:
import os
import shutil
import json
import re
import random
import sys

import nltk
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
!pip install https://download.pytorch.org/whl/cu101/torch-1.4.0-cp38-cp38-win_amd64.whl
!git clone  https://github.com/sberbank-ai/ru-gpts
!mkdir models2/

In [8]:
PATH_TO_DA_NET = '../da_net'
PATH_TO_RUBQ = '../rubq'

In [9]:
da_net_dataset = []
with open(f'{PATH_TO_DA_NET}/train.jsonl' , 'r', encoding='utf-8') as f:
    line = f.readline()
    while line:
        da_net_dataset.append(json.loads(line))
        line = f.readline()

In [10]:
for val in da_net_dataset:
    if 'label' in val:
        val.pop('label')
    if 'idx' in val:
        val.pop('idx')

In [11]:
with open(f'{PATH_TO_RUBQ}/RuBQ_2.0_dev.json', 'r', encoding='utf-8') as f:
    rubq = json.load(f)

In [12]:
with open(f'{PATH_TO_RUBQ}/RuBQ_2.0_paragraphs.json', 'r', encoding='utf-8') as p:
    paragraphs = json.load(p)

In [13]:
rubq_dataset = []
for q in rubq:
    item = {'question': '', 'passage': ''}
    item['question'] = q['question_text']
    for ans in q['paragraphs_uids']['with_answer']:
        p = paragraphs[ans]
        if p['uid'] != ans:
            print('bad')
        else:
            item['passage'] = p['text']
            rubq_dataset.append(item)
            item = {'question': q['question_text'], 'passage': ''}

In [14]:
dataset = []
dataset += rubq_dataset
dataset += da_net_dataset

In [15]:
val_set = []
rubq_val = rubq_dataset[-198:]
random.shuffle(rubq_val)
for d in rubq_val:
    val_set.append({"passage": f"<s>Текст: {d['passage']}\nВопрос:", "question": d['question']})
da_net_val = da_net_dataset[-198:]
random.shuffle(da_net_val)
for d in da_net_val:
    val_set.append({"passage": f"<s>Текст: {d['passage']}\nВопрос:", "question": d['question']})

In [246]:
val = []
random.shuffle(dataset)
with open('train_questions.txt', 'wb') as f:
    train = dataset[:3 * len(dataset) // 4]
    random.shuffle(train)
    for d in train:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))
with open('valid_questions.txt', 'wb') as f:
    val = dataset[3 * len(dataset) // 4:]
    random.shuffle(val)
    for d in val:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))

In [248]:
with open('train_questions_rubq.txt', 'wb') as f:
    for d in rubq_dataset[:3 * len(rubq_dataset) // 4]:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))
with open('valid_questions_rubq.txt', 'wb') as f:
    for d in rubq_dataset[3 * len(rubq_dataset) // 4:]:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))
        

In [249]:
with open('train_questions_da_net.txt', 'wb') as f:
    for d in da_net_dataset[:3 * len(da_net_dataset) // 4]:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))
with open('valid_questions_da_net.txt', 'wb') as f:
    for d in da_net_dataset[3 * len(da_net_dataset) // 4:]:
        f.write(f"<s>Текст: {d['passage']}\n".encode('utf8'))
        f.write(f"Вопрос: {d['question']}</s>\n".encode(sys.stdout.encoding, errors='replace'))
        

In [ ]:
!python ru-gpts/pretrain_transformers.py --output_dir=models/questions --model_type=gpt2 --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 --do_train --train_data_file=train_questions.txt --do_eval --eval_data_file=valid_questions.txt --per_gpu_train_batch_size 1 --per_gpu_eval_batch_size 1 --gradient_accumulation_steps 1 --num_train_epochs 5 --block_size 256 --overwrite_output_dir
!python ru-gpts/pretrain_transformers.py --output_dir=models/questions_rubq --model_type=gpt2 --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 --do_train --train_data_file=train_questions_rubq.txt --do_eval --eval_data_file=valid_questions_rubq.txt --per_gpu_train_batch_size 1 --per_gpu_eval_batch_size 1 --gradient_accumulation_steps 1 --num_train_epochs 5 --block_size 256 --overwrite_output_dir
!python ru-gpts/pretrain_transformers.py --output_dir=models/questions_da_net --model_type=gpt2 --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 --do_train --train_data_file=train_questions_da_net.txt --do_eval --eval_data_file=valid_questions_da_net.txt --per_gpu_train_batch_size 1 --per_gpu_eval_batch_size 1 --gradient_accumulation_steps 1 --num_train_epochs 5 --block_size 256 --overwrite_output_dir

In [21]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed

np.random.seed(42)
torch.manual_seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.manual_seed(42)
set_seed(42)

In [22]:
# hack for windows
import ctypes
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

<CDLL 'caffe2_nvrtc.dll', handle 7ff9de690000 at 0x25a18f3a7c0>

# DaNetQA + RuBQ

In [254]:
tok = GPT2Tokenizer.from_pretrained("models/questions")
model = GPT2LMHeadModel.from_pretrained("models/questions")
model.cuda()
i = 0
bleu_score = 0
for val in val_set:
    text = val['passage']
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(), max_length=256, repetition_penalty=5.0, do_sample=True, top_k=1, top_p=0.95, temperature=1)
    question = tok.decode(out[0])
    first_elem = question.find('Вопрос')
    last_elem = question.find('?')
    if last_elem > 0:
        question = question[first_elem + len(first_elem) + 1:last_elem + 1]
    else:
        question = question[first_elem + len(first_elem) + 1:]
    print(val['question'], question)
    cur_bleu_score = nltk.translate.bleu_score.sentence_bleu([word_tokenize(val['question'])], word_tokenize(question))
    i += 1
    bleu_score += cur_bleu_score
    print(cur_bleu_score)
avg_bleu = bleu_score / i

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда было построено первое метро?  Есть ли метро рядом со станциями джуманджи?
1.2882297539194154e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда был выпущен айфон 8?  Есть ли у айфона s8 g?
1.1200407237786664e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Разрешены ли в польше однополые браки?  Разрешено ли высшее образование за рубежом?
1.331960397810445e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Есть ли у медведя хвост?  Есть ли хвост?
8.17527103954383e-155


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Существует ли в природе чёрный цвет?  Есть ли у черного дерева характерный для данного типа окраски оттенок?
1.1640469867513693e-231
Был ли референдум при распаде ссср? 
0


In [255]:
avg_bleu

0.03968260136808063

# DaNetQA

In [256]:
tok = GPT2Tokenizer.from_pretrained("models/questions_da_net")
model = GPT2LMHeadModel.from_pretrained("models/questions_da_net")
model.cuda()
i = 0
bleu_score = 0
for val in val_set:
    text = val['passage']
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(), max_length=256, repetition_penalty=5.0, do_sample=True, top_k=1, top_p=0.95, temperature=1)
    question = tok.decode(out[0])
    first_elem = question.find('Вопрос')
    last_elem = question.find('?')
    if last_elem > 0:
        question = question[first_elem + 7:last_elem + 1]
    else:
        question = question[first_elem + 7:]
    print(val['question'], question)
    cur_bleu_score = nltk.translate.bleu_score.sentence_bleu([word_tokenize(val['question'])], word_tokenize(question))
    i += 1
    bleu_score += cur_bleu_score
    print(cur_bleu_score)
avg_bleu = bleu_score / i

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда было построено первое метро?  Был ли построен титаник?
9.97486269044271e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда был выпущен айфон 8?  Есть ли блютуз на телефоне?
1.1640469867513693e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Разрешены ли в польше однополые браки?  Разрешено ли во внутригосударственном порядке заниматься лесбиянством?
1.2882297539194154e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Есть ли у медведя хвост?  Есть ли хвост на медведе?
9.013778876140909e-155


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Существует ли в природе чёрный цвет?  Есть ли у черного/серого цветуха цветные характеристики?
1.2882297539194154e-231
Был ли референдум при распаде ссср? 
0


In [257]:
avg_bleu

0.019568637805097853

# RuBQ

In [258]:
tok = GPT2Tokenizer.from_pretrained("models/questions_rubq")
model = GPT2LMHeadModel.from_pretrained("models/questions_rubq")
model.cuda()
i = 0
bleu_score = 0
for val in val_set:
    text = val['passage']
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(), max_length=256, repetition_penalty=5.0, do_sample=True, top_k=1, top_p=0.95, temperature=1)
    question = tok.decode(out[0])
    first_elem = question.find('Вопрос')
    last_elem = question.find('?')
    if last_elem > 0:
        question = question[first_elem + 7:last_elem + 1]
    else:
        question = question[first_elem + 7:]
    print(val['question'], question)
    cur_bleu_score = nltk.translate.bleu_score.sentence_bleu([word_tokenize(val['question'])], word_tokenize(question))
    i += 1
    bleu_score += cur_bleu_score
    print(cur_bleu_score)
avg_bleu = bleu_score / i

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда было построено первое метро?  Когда был построен первый поезд?
1.384292958842266e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Когда был выпущен айфон 8?  Какой телефон поддерживает айфон?
1.1862177682648818e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Разрешены ли в польше однополые браки?  Когда был принят закон «О браке между мужчиной (женщиной)»?
9.418382295637229e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Есть ли у медведя хвост?  Какой медведь живет на территории Иркутской области?
1.0832677820940877e-231


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Существует ли в природе чёрный цвет?  Какой основной цветовой показатель у чёрного?
1.1200407237786664e-231
Был ли референдум при распаде ссср? 
0


In [259]:
avg_bleu

0.04790795542504064